In [1]:
# https://geobuf-cpp.readthedocs.io/en/latest/code/
# pip install -U pybind11-geobuf

from pybind11_geobuf import Encoder, Decoder, geojson, rapidjson, __version__

print(__version__)

0.0.9


In [2]:
def sample_geojson():
    geojson = {
        "type": "Feature",
        "properties": {
            "string": "string",
            "int": 42,
            "int2": -101,
            "double": 3.141592653,
            "list": ["a", "list", "is", "a", "list"],
            "dict": {"key": 42, "value": 3.14},
        },
        "geometry": {
            "coordinates": [
                [120.40317479950272, 31.416966084052177, 1.111111],
                [120.28451900911591, 31.30578266928819, 2.22],
                [120.35592249359615, 31.21781895672254, 3.3333333333333],
                [120.67093786630113, 31.299502266522722, 4.4],
            ],
            "type": "LineString",
            "extra_key": "extra_value",
        },
        "my_key": "my_value",
    }
    return geojson

In [3]:
g = geojson.Feature().from_rapidjson(sample_geojson())
print(g())

print(g.as_numpy())
print(g.geometry())

pbf = g.to_geobuf()
print(pbf)
gg = geojson.Feature().from_geobuf(pbf)
print(gg())
print(gg.as_numpy())

gg = geojson.Feature().from_geobuf(g.to_geobuf(precision=6))
print(gg.as_numpy())
gg = geojson.Feature().from_geobuf(g.to_geobuf(precision=6, only_xy=True))
print(gg.as_numpy())

{'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[120.40317479950272, 31.416966084052177, 1.111111], [120.28451900911591, 31.30578266928819, 2.22], [120.35592249359615, 31.21781895672254, 3.3333333333333], [120.67093786630113, 31.299502266522722, 4.4]], 'extra_key': 'extra_value'}, 'properties': {'dict': {'value': 3.14, 'key': 42}, 'list': ['a', 'list', 'is', 'a', 'list'], 'double': 3.141592653, 'int2': -101, 'int': 42, 'string': 'string'}, 'my_key': 'my_value'}
[[120.4031748   31.41696608   1.111111  ]
 [120.28451901  31.30578267   2.22      ]
 [120.35592249  31.21781896   3.33333333]
 [120.67093787  31.29950227   4.4       ]]
b'\n\x04dict\n\x06double\n\textra_key\n\x03int\n\x04int2\n\x04list\n\x06my_key\n\x06string\x10\x03\x18\x08*\xc1\x01\nI\x08\x02\x1a2\xd0\xa8\xc5\xdaY\xc0\xf1\x93\xb4\x17\xf8\xae\xfbi\xd5\xb7\xa8\x0b\x89\x9c\xcd\n\x88\x9f\xe0i\xf8\xcf\xe7\x06\xa5\xe3\xb1\x08\xaa\xbf\x96j\xc4\xb3\x85\x1e\x96\x8e\xe5\x07\xd6\xea\xdcej\r\n\x0bextra_valuez\x02\x0

In [4]:
decoder = Decoder()
with open('cloverleaf.pbf', 'rb') as f:
    encoded = f.read()
ret = decoder.decode(encoded)

print(type(ret), len(ret))
assert ret == decoder.decode(encoded)

<class 'str'> 365500


In [5]:
ret = decoder.decode_to_rapidjson(encoded)
print(type(ret), type(ret()))

<class '_pybind11_geobuf.rapidjson'> <class 'dict'>


In [6]:
geojson = decoder.decode_to_rapidjson(encoded)()
assert geojson['type'] == 'FeatureCollection'

In [7]:
assert 8 == decoder.precision()

In [8]:
encoder = Encoder(max_precision=10**8)
encoded = encoder.encode({
    "type": "Polygon",
    "coordinates": [[[100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0]]]
})

print(encoded)

from pybind11_geobuf import pbf_decode
print(pbf_decode(encoded))

b'\x18\x002\r\x08\x04\x1a\t\xc8\x01\x00\x02\x00\x00\x02\x01\x00'
3: 0
6: 
  1: 4
  3: 200,0,2,0,0,2,1,0



In [9]:
print(decoder.decode_to_rapidjson(encoded)())

{'type': 'Polygon', 'coordinates': [[[100.0, 0.0, 0.0], [101.0, 0.0, 0.0], [101.0, 1.0, 0.0], [100.0, 1.0, 0.0], [100.0, 0.0, 0.0]]]}
